# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования: Дать ответ на следующие четыре вопроса:**

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о кредитной статистике находяться в файле '/datasets/data.csv'. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Далее необходимо проверить данные на ошибки и оценить их влияние на исследование.

Затем, на этапе предобработки необходимо обработать пропущенные значения, при необходимости, заменить тип данных, обработать дубликаты, провести лемматизацию и категоризацию данных.

После подготовки данных, необходимо дать ответы на поставленные вопросы и сделать выводы.

**Таким образом, исследование пройдёт в четыре этапа:**

1. [Обзор данных, изучение общей информации](#start)
2. [Предобработка данных](#preprocessing)<br>
   2.1. [Обработка пропусков](#null)<br>
   2.2. [Замена типа данных](#change)<br>
   2.3. [Обработка дубликатов](#duplicates)<br>
   2.4. [Лемматизация](#lemma)<br>
  2.5. [Категоризация данных](#category)<br>
3. [Ответы на вопросы](#question)<br>
4. [Общий вывод](#conclusion)<br>



<a id ="start"></a>
## Обзор данных, изучение общей информации

In [2]:
# импорт библиотеки pandas
import pandas as pd

# Импортируем Counter
from collections import Counter

In [8]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('data.csv')

In [9]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Для более детального знакомства с данными выведем первые, последние и случайные 10 строк датафрейма:

In [10]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [11]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


In [12]:
df.sample(10, random_state = 19 ).T

,16619,12945,2451,8490,13294,6421,6746,16772,9355,8620
children,0,0,0,1,0,0,0,0,0,4
days_employed,NaN,NaN,-1521.923874,NaN,355815.850208,-428.033481,-4237.553184,-2177.391045,374762.301257,-2688.85142
dob_years,67,47,54,31,62,32,34,25,58,34
education,среднее,высшее,среднее,среднее,среднее,высшее,высшее,высшее,среднее,среднее
education_id,1,0,1,1,1,0,0,0,1,1
family_status,гражданский брак,женат / замужем,женат / замужем,женат / замужем,в разводе,гражданский брак,женат / замужем,женат / замужем,женат / замужем,женат / замужем
family_status_id,1,0,0,0,3,1,0,0,0,0
gender,M,M,F,F,F,M,F,F,F,F
income_type,пенсионер,сотрудник,сотрудник,компаньон,пенсионер,сотрудник,сотрудник,компаньон,пенсионер,сотрудник
debt,1,0,0,0,0,0,0,0,0,0


In [449]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


**Вывод**

Итак, в таблице 12 столбцов. Тип данных в столбцах — int64, float64, object.<br>

Согласно документации к данным:<br>

children — количество детей в семье<br>
days_employed — общий трудовой стаж в днях<br>
dob_years — возраст клиента в годах<br>
education — уровень образования клиента<br>
education_id — идентификатор уровня образования<br>
family_status — семейное положение<br>
family_status_id — идентификатор семейного положения<br>
gender — пол клиента<br>
income_type — тип занятости<br>
debt — имел ли задолженность по возврату кредитов<br>
total_income — ежемесячный доход<br>
purpose — цель получения кредита<br>

Количество значений в столбцах days_employed и total_income отличается. Значит, в данных есть пропущенные значения.<br>
В столбцах education, education_id, family_status, family_status_id, gender, income_type, debt, purpose - категориальные данные<br>
В столбцах days_employed, dob_years, total_income - количественные данные.<br>
Столбцы education - education_id и family_status - family_status_id еобходимо вынести в отдельную таблицу.<br>
В столбце days_employed некоректные (мусорные) данные. Согласно документации там должны быть целые значения в днях. Учитывая что данные в этом столбце не используются для анализа поставленных целей, запрашивать дополнительную информацию нет необходимости.  В целях данной работы, необходимо изменить тип данных с float64 на int64 и убрать минусы<br>

<a id="preprocessing"></a>
## Предобработка данных

<a id="null"></a>
### Обработка пропусков

Подсчет нулевых значений в таблице df

In [450]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные данные найдены в столбцах days_employed и total_income. Данные в этих столбцах - количественного типа.<br> 
Колличество пропущенных значений в столбцах  - 2174, что составляет более 10% выборки. Удаление строк с нулевыми значениями приведет к сокращению выборки и повлияет на результат исследования.<br>
Для замены нулевых значений в столбце days_employed будем использовать среднее значение трудового стажа с учетом группировки по типу занятости (столбец income_type)<br> 
Для замены нулевых значений в столбце total_income будем использовать медианное значение ежемесячного дохода с учетом группировки по типу занятости (столбец income_type)<br> 

Определение уникальных значений столбца income_type и их кол-во

In [451]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Определение количества нулевых значений столбца days_employed агрегированных по столбцу income_type

In [452]:
df[df['days_employed'].isna()].groupby('income_type')['dob_years'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: dob_years, dtype: int64

Определение количества нулевых значений столбца days_employed агрегированных по столбцу income_type

In [453]:
df[df['total_income'].isna()].groupby('income_type')['dob_years'].count()

income_type
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
Name: dob_years, dtype: int64

Определение средних значений столбца days_employed агрегированных по столбцу income_type

In [454]:
days_employed_mean = df.groupby('income_type').agg({'days_employed': 'mean'})
display(days_employed_mean)

,days_employed
income_type,
безработный,366413.652744
в декрете,-3296.759962
госслужащий,-3399.896902
компаньон,-2111.524398
пенсионер,365003.491245
предприниматель,-520.848083
сотрудник,-2326.499216
студент,-578.751554


Определение медианных значений столбца days_employed агрегированных по столбцу income_type

In [455]:
total_income_median = df.groupby('income_type').agg({'total_income': 'median'})
display(total_income_median)

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Замена нулевых значений в столбцах 'days_employed' и 'total_income'

In [456]:
types = df['income_type'].unique()
for value in types:
    df.loc[df.loc[:,'income_type'] == value,'days_employed'] =  df.loc[df.loc[:,'income_type'] == value, 'days_employed'].fillna(days_employed_mean.loc[value, 'days_employed'])
    df.loc[df.loc[:,'income_type'] == value,'total_income'] =  df.loc[df.loc[:,'income_type'] == value, 'total_income'].fillna(total_income_median.loc[value, 'total_income'])
    

Проверка удаления нулевых значений в таблице df

In [457]:
df.isna().sum()                                                                 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Проверка удаления нулевых значений в таблице df

In [458]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [459]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Нулевые значения в столбце total_income были заменены медианным значением данного столбца в разрезе типа занятости.<br>
Нулевые значения в столбце days_employed были заменены среднеим значением трудового стажа  в разрезе типа занятости(столбец income_type).

<a id="change"></a>
### Замена типа данных 

In [460]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,21525.0,63489.685315,141168.183775,-18388.949901,-2570.047544,-1355.683356,-316.240646,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,21525.0,165225.324514,98043.665491,20667.263793,107798.172619,142594.396847,195549.940861,2.265604e+06


Замена типа данных в столбцах:<br>
Исходя из max и min значений заменим типы данных в столбцах на следующие:
- 'days_employed' c float64 на int32
- 'children', 'dob_years', 'education_id', 'family_status_id' с int 32 на int8

In [461]:
df['days_employed'] = df['days_employed'].astype('int32')

In [462]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,21525.0,63490.008688,141167.799840,-18388.000000,-2570.000000,-1355.000000,-316.000000,4.017550e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,21525.0,165225.324514,98043.665491,20667.263793,107798.172619,142594.396847,195549.940861,2.265604e+06


In [463]:
df[['children', 'dob_years', 'education_id', 'family_status_id']] = df[['children', 'dob_years', 'education_id', 'family_status_id']].astype('int8')

In [464]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int32
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int32(1), int64(1), int8(4), object(5)
memory usage: 1.3+ MB


Приведение к абсолютным значениям данных в столбце 'days_employed'

In [465]:
df['days_employed'] = df['days_employed'].abs()

**Вывод**

Типы данных в столбцах на следующие:
- 'days_employed' c float64 на int32
- 'children', 'dob_years', 'education_id', 'family_status_id' с int 32 на int8

<a id="duplicates"></a>
### Обработка дубликатов

Обработку дубликатов будем проводить в 2 этапа:
1. Поиск явных дубликатов
2. Поиск неявных дубликатов

#### Поиск явных дубликатов

Подсчет явных дубликатов

In [466]:
df.duplicated().sum()

54

Удаление явных дубликатов

In [467]:
df = df.drop_duplicates().reset_index(drop=True)

Проверка удаления явных дубликатов

In [468]:
# Проверка удаления явных дубликатов
df.duplicated().sum()

0

####  Поиск неявных дубликатов

При выводе 10 строк таблицы df видно что в столбце education есть неявные дубликаты. Исправим их.

**Проверка столбца столбца education**

In [469]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Приведение всех значений к нижнему регистру

In [470]:
df['education'] = df['education'].str.lower()

вывод уникальных значений столбца education

In [471]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Проверка столбца family_status**

In [472]:

df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Приведение всех значений к нижнему регистру

In [474]:
df['family_status'] = df['family_status'].str.lower()

In [475]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

**Проверка столбца income_type**

In [476]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

**Проверка столбца income_type**


In [477]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Подсчет дубликатов после форматирования данных**

In [478]:
df.duplicated().sum()

17

**Удаление явных дубликатов после форматирования данных**

In [479]:
df = df.drop_duplicates().reset_index(drop=True)

In [480]:
df.duplicated().sum()

0

**В целях оптимизации вынесем столбцы education - education_id и family_status - family_status_id в отдельны датафреймы, оставив education_id и family_status_id**

In [481]:
family_status = df[['family_status_id','family_status']].drop_duplicates().reset_index(drop = True)

In [482]:
education = df[['education_id','education']].drop_duplicates().reset_index(drop = True)

In [483]:
family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


In [484]:
education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


**Удалим столбцы education и family_status_id из таблицы df**

In [485]:
df = df.drop(columns=['family_status', 'education'])

In [486]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 10 columns):
children            21454 non-null int8
days_employed       21454 non-null int32
dob_years           21454 non-null int8
education_id        21454 non-null int8
family_status_id    21454 non-null int8
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int32(1), int64(1), int8(4), object(3)
memory usage: 1005.8+ KB


**Вывод**

Поиском явных дубликатов было обнаружено и удалено 54 дубликата,<br>
После обработки неявных дубликатов было обнаружено и удалено еще 17 дубликатов.<br>
Возможные причины возникновения дубликатов:
- техническая ошибка при формировании данных
- совпадение данных у разных клиентов, но тк уникальных id датафрейм несодержит, считаем что то техническая ошибка. 

В целях птимизации таблицы, вынесем столбцы family_status и education в отдельные датафреймы

### Лемматизация <a id="lemma"></a>

В меющемся датафрейме лемматизацию целесообразно проводить только со столбцом purpose

In [487]:
# Добавление к датафрейму дополнительного столбца, лемматизация столбца purpose
df['lemmas'] = df['purpose'].apply(m.lemmatize)

In [488]:
# Добавление к датафрейму дополнительного столбца, counter
df['lemmas_counter'] = df['lemmas'].apply(Counter)

In [489]:
# Вывод уникальных значений counter
df['lemmas_counter'].value_counts()

{'автомобиль': 1, '
': 1}                                                        972
{'свадьба': 1, '
': 1}                                                           791
{'на': 1, ' ': 2, 'проведение': 1, 'свадьба': 1, '
': 1}                         768
{'сыграть': 1, ' ': 1, 'свадьба': 1, '
': 1}                                     765
{'операция': 1, ' ': 2, 'с': 1, 'недвижимость': 1, '
': 1}                       675
{'покупка': 1, ' ': 2, 'коммерческий': 1, 'недвижимость': 1, '
': 1}             661
{'операция': 1, ' ': 2, 'с': 1, 'жилье': 1, '
': 1}                              652
{'покупка': 1, ' ': 3, 'жилье': 1, 'для': 1, 'сдача': 1, '
': 1}                 651
{'операция': 1, ' ': 3, 'с': 1, 'коммерческий': 1, 'недвижимость': 1, '
': 1}    650
{'покупка': 1, ' ': 1, 'жилье': 1, '
': 1}                                       646
{'жилье': 1, '
': 1}                                                             646
{'покупка': 1, ' ': 3, 'жилье': 1, 'для': 1, 'семья': 1, '
': 1} 

**Вывод**

Лемитизация выполнена, основные слова: жилье, недвижимость, автомобиль, образование, свдьба

### Категоризация данных <a id="category"></a>

#### Категоризация данных в столбце children

Для ответа на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?" категоризуем данные в столбце  children по принципу:
- 0 детей - бездетные
- от 1 до 2-х детей - с детьми
- от 3-х детей - многодетные

In [490]:
df['children'].value_counts()

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце обнаружены неадекватные значения: -1 и 20, скорее всго опечатка. Изменим -1 на 1 а 20 на 2

In [491]:
df['children'] = df['children'].replace(-1, 1)

In [492]:
df['children'] = df['children'].replace(20, 2)

In [493]:
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Функция, для категоризации children:

In [494]:
def childrens_category (number):
    if number == 0:
        return('Бездетный')
    if 0 < number < 3:
        return('C детьми')
    else:
        return('Многодетный')

Добавим в датафрейм категории наличия детей:

In [495]:
df['children_category'] = df['children'].apply(childrens_category)

In [496]:
df[['children', 'children_category']].head(10)

,children,children_category
0,1,C детьми
1,1,C детьми
2,0,Бездетный
3,3,Многодетный
4,0,Бездетный
5,0,Бездетный
6,0,Бездетный
7,0,Бездетный
8,2,C детьми
9,0,Бездетный


#### Категоризация данных в столбце family_status_id

Для ответа на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?" категоризуем данные в столбце family_status_id и соединим полученные данные с таблицей family_status по принципу:
  - Семейные: 'женат / замужем', 'гражданский брак'       
  - Холостые: 'вдовец / вдова', 'в разводе', 'не женат / не замужем'

In [497]:
family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Функция, для категоризации family_status:

In [498]:
def family_category (status):
    family = [0, 1]
    single = [2, 3, 4]
    if status in family:
        return('Семейные')
    if status in single:
        return('Холостые')

Добавим в датафрейм категории семейного положения:

In [499]:
df['family_category'] = df['family_status_id'].apply(family_category)

In [500]:
df[['family_status_id', 'family_category']].head(10)

,family_status_id,family_category
0,0,Семейные
1,0,Семейные
2,0,Семейные
3,0,Семейные
4,1,Семейные
5,1,Семейные
6,0,Семейные
7,0,Семейные
8,1,Семейные
9,0,Семейные


#### Категоризация данных в столбце total_income

Для ответа на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?" категоризуем данные в столбце total_income по принципу:

 - ниже среднего
 - выше среднего
 
Для более точной картины, категоризацию проведем по медианному значению в разрезе типа занятости (столбец income_type)

Вывод медианного значения уровня дохода:

In [501]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Функция, для категоризации total_income:

In [502]:

def income_category (row):
    income_type = row['income_type']
    total_income = row['total_income']
    
    if income_type == 'безработный': 
        if total_income < 131339.751676:
            return 'ниже среднего'
    if income_type == 'в декрете': 
        if total_income < 53829.130729:
            return 'ниже среднего'
    if income_type == 'госслужащий': 
        if total_income < 150447.935283:
            return 'ниже среднего'
    if income_type == 'компаньон': 
        if total_income < 172357.950966:
            return 'ниже среднего'
    if income_type == 'пенсионер': 
        if total_income < 118514.486412:
            return 'ниже среднего'
    if income_type == 'предприниматель': 
        if total_income < 499163.144947:
            return 'ниже среднего'
    if income_type == 'сотрудник': 
        if total_income < 142594.396847:
            return 'ниже среднего'
    if income_type == 'студент': 
        if total_income < 98201.625314:
            return 'ниже среднего'
    return 'выше среднего'


Добавим в датафрейм категории семейного положения:

In [503]:
df['income_category'] = df.apply(income_category, axis=1)

In [504]:
df[['income_type', 'total_income', 'income_category']].head(10)

,income_type,total_income,income_category
0,сотрудник,253875.639453,выше среднего
1,сотрудник,112080.014102,ниже среднего
2,сотрудник,145885.952297,выше среднего
3,сотрудник,267628.550329,выше среднего
4,пенсионер,158616.077870,выше среднего
5,компаньон,255763.565419,выше среднего
6,компаньон,240525.971920,выше среднего
7,сотрудник,135823.934197,ниже среднего
8,сотрудник,95856.832424,ниже среднего
9,сотрудник,144425.938277,выше среднего


#### Категоризация данных в столбце purpose

Для ответа на вопрос "Как разные цели кредита влияют на его возврат в срок?" категоризуем данные используя выполненную ранее лематезацию
Основные категории цели получения кредита:
 - операции с жилой недвижимостью
 - операции с коммерческой недвижимостью
 - образование
 - покупка автомобиля
 - проведение свадьбы
 


Функция, для категоризации purpose:

In [505]:
def purpose_category (row):
    
    if 'жилье' in row:
        return('операции с жилой недвижимостью')
    if 'недвижимость' in row:
        if ['собственный', 'свой', 'жилой'] in row:
            return('операции с жилой недвижимостью')
        return('операции с коммерческой недвижимостью')
    if 'образование' in row:
        return('образование')
    if 'автомобиль' in row:
        return('покупка автомобиля')
    if 'свадьба' in row:
        return('проведение свадьбы')

Добавление в датафрейм категории целей кредита

In [506]:
df['purpose_category'] = df['lemmas'].apply(purpose_category)

In [507]:
# Проверка добавления нового столбца
df[['purpose', 'purpose_category']].head(10)

,purpose,purpose_category
0,покупка жилья,операции с жилой недвижимостью
1,приобретение автомобиля,покупка автомобиля
2,покупка жилья,операции с жилой недвижимостью
3,дополнительное образование,образование
4,сыграть свадьбу,проведение свадьбы
5,покупка жилья,операции с жилой недвижимостью
6,операции с жильем,операции с жилой недвижимостью
7,образование,образование
8,на проведение свадьбы,проведение свадьбы
9,покупка жилья для семьи,операции с жилой недвижимостью


**Вывод**

Для анализа данных и ответов на поставленные вопросы была проведена категоризация необходимых столбцов.

## Ответьте на вопросы <a id="question"></a>

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Подсчет конверсии невозвратов в разрезе груп наличия детей

In [508]:
df.groupby('children_category')['debt'].sum()/df.groupby('children_category')['debt'].count()

children_category
C детьми       0.092654
Бездетный      0.075438
Многодетный    0.081579
Name: debt, dtype: float64

**Вывод**

Полученные значения конверсии невозврата кредита в 3-х категориях:
- С детьми - 9,2%
- Бездетные - 7,5%
- Многодетные - 8,1%

Наименьший процент невозврата кредита в срок наблюдается у клиентов без детей.<br>
Клиенты имеющие одного или двоих детей имеют наибольший проент невозврата кредита в срок.<br>
Многодетные клиенты, имеющие более 3-х детей, имеют средний показатель невозврата кредита в срок.<br>


**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Подсчет конверсии невозвратов в разрезе категории семейного положения


In [509]:
df.groupby('family_category')['debt'].sum()/df.groupby('family_category')['debt'].count()

family_category
Семейные    0.079988
Холостые    0.085012
Name: debt, dtype: float64

**Вывод**

Полученные значения конверсии невозврата кредита сопоставимы в 2-х категориях:
 - Семейные - 8%
 - Холостые - 8,5%

Можно сделать вывод что семейное положение не сильно влияет на возврат кредита в срок (в пределах 0,5%).

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Подсчет конверсии невозвратов в разрезе категории уровня дохода

In [510]:
df.groupby(['income_type', 'income_category'])['debt'].sum()/df.groupby(['income_type', 'income_category'])['debt'].count()

income_type      income_category
безработный      выше среднего      0.000000
                 ниже среднего      1.000000
в декрете        выше среднего      1.000000
госслужащий      выше среднего      0.049938
                 ниже среднего      0.070122
компаньон        выше среднего      0.070367
                 ниже среднего      0.077061
пенсионер        выше среднего      0.056363
                 ниже среднего      0.056452
предприниматель  выше среднего      0.000000
сотрудник        выше среднего      0.093138
                 ниже среднего      0.098862
студент          выше среднего      0.000000
Name: debt, dtype: float64

Вывод уникальных значений столбца 'income_type'

In [511]:
df['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Для категорий: в декрете, студент, безработный и предприниматель количество наблюдений очень мало для анализа и формирования правильных выводов, поэтому анализ данных категорий производить не будем.

In [512]:
#Формирование сводной таблицы
column = ['уровень дохода', 'госслужащий', 'компаньон', 'пенсионер', 'сотрудник']
row = [['выше среднего', '4,9%', '7%', '5,6%', '9.3%'], ['ниже среднего', '7%', '7,7%', '5,6%', '9.8%']]
result = pd.DataFrame(data=row, columns=column)
display (result)

,уровень дохода,госслужащий,компаньон,пенсионер,сотрудник
0,выше среднего,"4,9%",7%,"5,6%",9.3%
1,ниже среднего,7%,"7,7%","5,6%",9.8%


**Вывод**

В категории "госслужащий" разница между не возвратом кредита в срок в зависимости от уровня дохода составляет 2%<br>
В категории "компаньон" разница между не возвратом кредита в срок в зависимости от уровня дохода составляет 0,7%<br>
В категории "пенсионер" разницы между не возвратом кредита в срок в зависимости от уровня дохода не наблюдается<br>
В категории "компаньон" разница между не возвратом кредита в срок в зависимости от уровня дохода составляет 0,5%<br>

При этом наблюдается общая тенденция не возврата кредита в срок в зависимости от типа занятости по всем уровням дохода:
пенсионеры (5,6%) - госслужащие (7%) - компаньон(7,7%) - сотрудник (9,8%)


**- Как разные цели кредита влияют на его возврат в срок?**

Подсчет конверсии невозвратов в разрезе категории цели кредита

In [513]:
df.groupby('purpose_category')['debt'].sum()/df.groupby('purpose_category')['debt'].count()

purpose_category
образование                              0.092200
операции с жилой недвижимостью           0.069058
операции с коммерческой недвижимостью    0.074634
покупка автомобиля                       0.093590
проведение свадьбы                       0.080034
Name: debt, dtype: float64

**Вывод**

Полученные значения конверсии невозврата кредита в срок в зависимости от цели кредита:
- образование - 9,2%
- операции с жилой недвижимостью - 6,9%
- операции с коммерческой недвижимостью - 7,4%
- покупка автомобиля  - 9,4%
- проведение свадьбы - 8%

Наименьший процент невозврата кредита в срок наблюдается у клиентов оформляющих кредит в целях покупки недвижимости (6,9% жилой и 7,2% коммерческой).<br>
Наибольший процент невозврата кредита в срок наблюдается у клиентов оформляющих кредит в целях покупки автомобиля и оплаты образования.<br>
Клиенты, оформляющие кредит в целях проведения свдьбы,невозвращают кредит в срок в 8% случаев.<br>


## Общий вывод <a id="conclusion"></a>

**Есть ли зависимость между наличием детей и возвратом кредита в срок?**<br>
Ответ: Зависимость есть:
 - наименьший процент невозврата кредита у клиентов без детей - 7,5%
 - наибольший процент невозврата кредита у клиентов с 1-м или 2-мя детьми - 9,2%<br>

**Есть ли зависимость между семейным положением и возвратом кредита в срок?**<br>
Ответ: Зависимость маловыраженная, семейные клиенты невозвращаюь кредит в срок на 0,5% реже
- процент невозврата кредита у семейных клиентов - 8%
- процент невозврата кредита у холостых клиентов - 8,5%<br>

**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**<br>
Ответ: Зависимость есть:
- наименьший процент невозврата кредита имеет категория клиентов "Госслужащий с уровнем дохода выше среднего" - 4,9 %
- наибольший процент у категории "Сотрудник с уровнем дохода ниже среднего" - 9,8%

**Как разные цели кредита влияют на его возврат в срок?**<br>
Ответ: 
- Наименьший процент невозврата кредита наблюдается у клиентов, оформляющих кредит в целях покупки жилой недвижимости. - 6,9%
- Наибольший процент невозврата кредита наблюдается у клиентов, оформляющих кредит в целях покупки автомобиля и оплаты образования - 9,3%.




## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.